## Disclaimer:

### O notebook em questão possui como objetivo a vizualisação e exploração de dados dos estudantes da Universidade Federal de Lavras (UFLA). Os dados em questão estão disponíveis para consulta pública [aqui](https://dados.gov.br/dados/conjuntos-dados/estudantes-da-graduacao). A ferramenta utilizada para manipulação dos dados foi o `PySpark`.

In [60]:
# Criação da sessão Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("data_exploration").getOrCreate()
import warnings
warnings.filterwarnings('ignore')

### SETUP

In [75]:
# IMPORTS
import pyspark.pandas as ps
import matplotlib.pyplot as plt

### DATA VISUALIZATION

In [63]:
# Carregando os dados como um objeto PySpark
data = spark.read.csv('alunos_2024.csv', header=True, inferSchema=True)
type(data)

pyspark.sql.dataframe.DataFrame

In [64]:
# Tabela com os dados
data.show()

+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+------------------+----------------+-----------------+--------------------+--------------+--------------+--------------+--------------------+
|registro_academico|               curso|semestre_letivo_referencia|ano_letivo_referencia|situacao_estudante|semestre_letivo_ingresso|forma_ingresso|forma_ingresso_tipo|codigo_oferta_curso|codigo_inep|curso_turno|nome_matriz_curricular|  titulo_academico|curso_modalidade|    situacao_nome|  situacao_descricao|situacao_censo|        campus|sistema_origem| data_atualizacao_dw|
+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+--------

In [65]:
# Schema dos dados
data.printSchema()

root
 |-- registro_academico: integer (nullable = true)
 |-- curso: string (nullable = true)
 |-- semestre_letivo_referencia: string (nullable = true)
 |-- ano_letivo_referencia: date (nullable = true)
 |-- situacao_estudante: string (nullable = true)
 |-- semestre_letivo_ingresso: string (nullable = true)
 |-- forma_ingresso: string (nullable = true)
 |-- forma_ingresso_tipo: string (nullable = true)
 |-- codigo_oferta_curso: string (nullable = true)
 |-- codigo_inep: integer (nullable = true)
 |-- curso_turno: string (nullable = true)
 |-- nome_matriz_curricular: integer (nullable = true)
 |-- titulo_academico: string (nullable = true)
 |-- curso_modalidade: string (nullable = true)
 |-- situacao_nome: string (nullable = true)
 |-- situacao_descricao: string (nullable = true)
 |-- situacao_censo: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- sistema_origem: string (nullable = true)
 |-- data_atualizacao_dw: timestamp (nullable = true)



In [40]:
# Para melhor manipulação trasnforma o objeto em SparkPandas
data = ps.DataFrame(data)
type(data)

pyspark.pandas.frame.DataFrame

In [41]:
# Para plotar os gráficos no notebook
%matplotlib inline

In [78]:
view_1 = ps.DataFrame(data.groupBy('semestre_letivo_referencia', 'situacao_descricao')
                      .count()
                      .orderBy('semestre_letivo_referencia'))

view_1_pivot = view_1.pivot(index='semestre_letivo_referencia', 
                            columns='situacao_descricao', 
                            values='count')
view_1_pivot.plot(kind='bar')

In [59]:
# Encerra a sessão spark
spark.stop()